In [1]:
from src import Config
from src.text_index import IndexReader

config = Config()

ir = IndexReader(config)

Top candidate similarity is calculated accurately to allow for setting a threshold of relevant/irrelevant results.
For example: if `sim` drops below 0.6, we may want to indicate in the interface that we are not certain of the result.

The scores for candidates are only useful for ranking, their absolute values are not that meaningful.
They are scaled so that the top result has score = 1.

In [2]:
sim, df = ir.query_monographs('Instytut badań Systemowawych polskiej akademii Nauk')
print(f'Top candidate similarity: {sim}')
df

Top candidate similarity: 0.9472589035614246


,score,name,domains_boost
id,,,
197,1.000000,Instytut Badań Systemowych Polskiej Akademii Nauk,1
309,0.613143,Instytut Systematyki i Ewolucji Zwierząt Polsk...,1
196,0.581534,Instytut Badań Literackich Polskiej Akademii Nauk,1
228,0.548672,Instytut Fizyki Polskiej Akademii Nauk,1
310,0.548672,Instytut Sztuki Polskiej Akademii Nauk,1


In [8]:
sim, df = ir.query_monographs('fffffffffffff')
print(f'Top candidate similarity: {sim}')
df

Top candidate similarity: 0


""


In [4]:
sim, df = ir.query_monographs('IBS PAN')
print(f'Top candidate similarity: {sim}')
df

Top candidate similarity: 0.5945165945165946


,score,name,domains_boost
id,,,
331,1.000000,John Benjamins Publishing Company,1
624,0.966923,William B. Eerdmans Publishing Company,1


In [5]:
sim, df = ir.query_journals('annals of nauki leśne', ['nauki leśne'])
print(f'Top candidate similarity: {sim}')
df

Top candidate similarity: 0.8476190476190476


,score,name,domains_boost
id,,,
1215,1.000000,ANNALS OF BOTANY,1.7
29823,0.990522,Leśne Prace Badawcze,1.7
20891,0.637266,Annals of GIS,1.0
29580,0.637266,Annals of PDE,1.0
27281,0.598468,Annals of Joint,1.0


In [6]:
sim, df = ir.query_journals('jeszcze bardziej bez związku z niczem', ['nauki leśne'])
print(f'Top candidate similarity: {sim}')
df

Top candidate similarity: 0.4917186417186417


,score,name,domains_boost
id,,,
29876,1.000000,TEKA Komisji Polsko-Ukraińskich Związków Kultu...,1
29813,0.853689,FORUM PRAWNICZE,1
29828,0.830858,Dzieje Najnowsze,1
29880,0.813664,Studia Prawnicze KUL,1
30052,0.813664,Zeszyty Prawnicze,1


In [7]:
sim, df = ir.query_conferences('International Conference on the Applications of Nauki Leśne in Computing')
print(f'Top candidate similarity: {sim}')
df


Top candidate similarity: 0.8955820921075158


,score,name,domains_boost
id,,,
778,1.000000,International Conference on Grid Computing and...,1
880,0.950556,International Conference on Pervasive Computin...,1
963,0.935159,International Conference on Worldwide Computin...,1
1159,0.905227,International Workshop on Soft Computing Appli...,1
1663,0.903003,IEEE International Conference on Service-Orien...,1
